<a href="https://colab.research.google.com/github/Kimjibeom/example/blob/main/%ED%83%80%EC%8A%88_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 라이브러리 import
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

data = pd.DataFrame(columns={'스테이션명','위치','상태정보','위도','경도'})

# 타슈 사이트 http://www.tashu.or.kr/userStationAction.do?process=list

url_tashu= "http://www.tashu.or.kr/userStationAction.do?process=list"

# 페이지 이동시 추가로 적어줘야되는 url
url_add = "&page="

# 전체 페이지 수
total_page = 27

# 27 page에 대한 정보를 for문 사용
for index in range(1, total_page+1) :
  if index == 1:
    url = url_tashu
  else :
    url = url_tashu + url_add + str(index)
  # url에서 HTML로 가져오기
  html = urlopen(url)

  #HTML을 parsing하여 변환
  bsObject =BeautifulSoup(html, "html.parser", from_encoding='utf-8')

  # history_table이라는 class를 가지 table만 찾기
  table = bsObject.find_all('table', {'class', 'history_table'})

  #table 내에서 tr 찾기
  tr = table[0].find_all('tr')

  #첫 tr은 contents가 아니라 데이터 정보를 의미하여 제외 [1:tr수까지]
  tr = tr[1:len(tr)]

  #한 페이지의 정보 crawling
  for index_tr in range(0,len(tr)) :
    td = tr[index_tr].find_all('td')

    # 장소
    station = td[0].text.split('.')[1] # 1. 무역전시관입구(택시승강장)

    # 위치 
    location = td[1].text

    # 상태
    condition = td[2].text

    # 좌표추출
    geo = td[3].a.attrs['href'].split('(')[1] # '36.374389', '127.392375');" 추출

    # 위도, 경도 추출을 위한 데이터 클렌징
    geo = re.sub('[;)]','',geo) # '36.374389', '127.392375');" -> '36.374389', '127.392375'
    detail_location = geo.replace("'","")   #작은 따옴표 삭제

    # 위도 Latitiude
    lat = detail_location.split(',')[0]

    # 경도 Longitude
    lon = detail_location.split(',')[1]

    # data에 크롤링 결과 저장
    data = data.append({'스테이션명':station,
                        '위치': location,
                        '상태정보' : condition,
                        '위도' : lon},
                       ignore_index=True)
    
    print('Complete of ' + str(index_tr+1) + ' information with' + str(index) + 'Page')
  
  # Crawling 막히는 것 방지
  time.sleep(1)

print('------------------------------------------------------------------------')
print(data)






Complete of 1 information with1Page
Complete of 2 information with1Page
Complete of 3 information with1Page
Complete of 4 information with1Page
Complete of 5 information with1Page
Complete of 6 information with1Page
Complete of 7 information with1Page
Complete of 8 information with1Page
Complete of 9 information with1Page
Complete of 10 information with1Page
Complete of 1 information with2Page
Complete of 2 information with2Page
Complete of 3 information with2Page
Complete of 4 information with2Page
Complete of 5 information with2Page
Complete of 6 information with2Page
Complete of 7 information with2Page
Complete of 8 information with2Page
Complete of 9 information with2Page
Complete of 10 information with2Page
Complete of 1 information with3Page
Complete of 2 information with3Page
Complete of 3 information with3Page
Complete of 4 information with3Page
Complete of 5 information with3Page
Complete of 6 information with3Page
Complete of 7 information with3Page
Complete of 8 information 

크롤링 저장하기

In [6]:
data.to_csv('TASHU.csv', encoding='utf-8-sig')